In [4]:
import itertools
import numpy as np
from multiprocessing.dummy import Pool
import os.path
import mpu
import time
import sys
sys.path.append('../')
from card import Card
import pickle

In [ ]:
if os.path.isfile('SevenCardHands.npy'):
    SevenCardHands = np.load('SevenCardHands.npy')
else:
    SevenCardHands = np.array(list(itertools.combinations(range(28),7)))
    np.save('SevenCardHands.npy', SevenCardHands)
print(SevenCardHands.shape)

In [ ]:
if os.path.isfile('SixCardHands.npy'):
    SixCardHands = np.load('SixCardHands.npy')
else:
    SixCardHands = np.array(list(itertools.combinations(range(28),6)))
    np.save('SixCardHands.npy', SixCardHands)
print(SixCardHands.shape)

In [ ]:
if os.path.isfile('FiveCardHands.npy'):
    FiveCardHands = np.load('FiveCardHands.npy')
else:
    FiveCardHands = np.array(list(itertools.combinations(range(28),5)))
    np.save('FiveCardHands.npy', FiveCardHands)
print(FiveCardHands.shape)

In [8]:
if os.path.isfile('TwoCardHands.npy'):
    TwoCardHands = np.load('TwoCardHands.npy')
else:
    TwoCardHands = np.array(list(itertools.combinations(range(28),2)))
    np.save('TwoCardHands.npy', TwoCardHands)
print(TwoCardHands.shape)

(378, 2)


In [ ]:
def score_hand(hand):
    flush = hand.sum(axis=0).sum(axis=1)
    is_flush = np.max(flush) >= 5
    card_values = hand.sum(axis=0).sum(axis=0)
    count = 0
    hand_high = 8
    is_stright = False
    stright_high = 5
    for i, s in enumerate(card_values): 
        if s >= 1:
            count += 1
            hand_high = i
            if count >=5:
                stright_high = i
                is_stright = True
        else:
            count = 0
    if is_stright and is_flush:
        return 8, stright_high, hand_high    
    is_four_of_a_kind = np.isin(card_values, 4)
    if np.sum(is_four_of_a_kind):
        return 7, np.argwhere(is_four_of_a_kind==True)[0][0], hand_high
    is_three_of_a_kind = np.isin(card_values, 3)
    is_pair = np.isin(card_values, 2)
    if np.sum(is_pair) and np.sum(is_three_of_a_kind):
        return 6, np.argwhere(is_three_of_a_kind==True)[0][0], hand_high
    if is_flush:
        return 5, np.argwhere(hand.sum(axis=0)[np.argmax(flush)]==1)[-1][0], hand_high
    if is_stright:
        return 4, stright_high, hand_high
    if np.sum(is_three_of_a_kind):
        return 3, np.argwhere(is_three_of_a_kind==True)[0][0], hand_high
    if np.sum(is_pair) >= 2:
        return 2, np.argwhere(is_pair==True)[1][0], hand_high
    if np.sum(is_pair):
        return 1, np.argwhere(is_pair==True)[0][0], hand_high
    return 0, 0, hand_high

In [ ]:
HandScores = {}
def run(k):
    hand = np.zeros((7, 28), dtype=np.int8)
    for i in range(7):
        hand[i,SevenCardHands[k,i]] = 1
    hand = hand.reshape(7,4,7)
    HandScores[SevenCardHands[k,:].tobytes()] = score_hand(hand)
p = Pool(10)
p.map(run, range(SevenCardHands.shape[0]))
p.close()
p.join()
mpu.io.write('HandScores.pickle', HandScores)

In [ ]:
HandScores = mpu.io.read('filename.pickle')

In [ ]:
scores = []
for hand in SevenCardHands:
    if np.all(np.in1d(FiveCardHands[0], hand)):
        scores.append(HandScores[hand.tobytes()])

In [ ]:
len(SevenCardHands) / 60 

In [6]:

def score_hands(index):
    hand = FiveCardHands[int(index)]
    histo = np.zeros((9))
    deck = np.arange(28)
    deck = np.delete(deck, hand)
    for i in range(iters):
        np.random.shuffle(deck)
        cards = np.zeros((num_cards,28))
        for k, c in enumerate(hand):
            cards[k,c] = 1
        for k in range(len(hand),num_cards):
            cards[k,deck[k]] = 1
        cards = cards.reshape(num_cards,4,7)
        histo[score_hand(cards)[0:1]] += 1
    score = 0
    for i in range(9):
        score += (i+1)*histo[i]/iters
    scores[hand.tobytes()] = score

scores = {}
num_cards = 7
iters_ = [10, 100, 1000]
for iter in iters_:
    iters  = iter
    p = Pool(10)
    p.map(score_hands, range(FiveCardHands.shape[0]))
    p.close()
    p.join()
    mpu.io.write('5_card_values_'+str(iter)+'.pickle', scores)

In [7]:
def score_hands(index):
    hand = SixCardHands[int(index)]
    histo = np.zeros((9))
    deck = np.arange(28)
    deck = np.delete(deck, hand)
    for i in range(iters):
        np.random.shuffle(deck)
        cards = np.zeros((num_cards,28))
        for k, c in enumerate(hand):
            cards[k,c] = 1
        for k in range(len(hand),num_cards):
            cards[k,deck[k]] = 1
        cards = cards.reshape(num_cards,4,7)
        histo[score_hand(cards)[0:1]] += 1
    score = 0
    for i in range(9):
        score += (i+1)*histo[i]/iters
    scores[hand.tobytes()] = score


scores = {}
num_cards = 7
iters_ = [10, 100, 1000]
for iter in iters_:
    iters  = iter
    p = Pool(5)
    p.map(score_hands, range(SixCardHands.shape[0]))
    p.close()
    p.join()
    mpu.io.write('6_card_values_'+str(iter)+'.pickle', scores)

In [9]:
def score_hands(index):
    hand = TwoCardHands[int(index)]
    histo = np.zeros((9))
    deck = np.arange(28)
    deck = np.delete(deck, hand)
    for i in range(iters):
        np.random.shuffle(deck)
        cards = np.zeros((num_cards,28))
        for k, c in enumerate(hand):
            cards[k,c] = 1
        for k in range(len(hand),num_cards):
            cards[k,deck[k]] = 1
        cards = cards.reshape(num_cards,4,7)
        histo[score_hand(cards)[0:1]] += 1
    score = 0
    for i in range(9):
        score += (i+1)*histo[i]/iters
    scores[hand.tobytes()] = score


scores = {}
num_cards = 7
iters_ = [10, 100, 1000]
for iter in iters_:
    iters  = iter
    p = Pool(5)
    p.map(score_hands, range(TwoCardHands.shape[0]))
    p.close()
    p.join()
    mpu.io.write('2_card_values_'+str(iter)+'.pickle', scores)

In [6]:
for i in [10,100,1000]:
    dic = mpu.io.read('2_card_values_' + str(i) + '.pickle')
    dic.update(mpu.io.read('5_card_values_' + str(i) + '.pickle'))
    dic.update(mpu.io.read('6_card_values_' + str(i) + '.pickle'))
    mpu.io.write('hand_value_table_'+str(i)+'.pickle', dic)